## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
import numpy as np
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from PIL import Image
from copy import deepcopy
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from efficientnet_pytorch import EfficientNet

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Set random seed
SEED = 2021
# random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [4]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'

In [5]:
## HYPER PARAMETER 정의
EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.001
CLASS_NUM = 18
IMAGE_SIZE = 380
CROP_IMAGE_SIZE = 224
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is using!')

cuda:0 is using!


## 1. Pretrained Model 불러오기

In [6]:
efficientnet = EfficientNet.from_pretrained('efficientnet-b4', num_classes=18)
print('네트워크 필요 입력 채널 개수', efficientnet._conv_stem.weight.shape[1])
print('네트워크 출력 채널 개수 (예측 class type 개수)', efficientnet._fc.weight.shape[0])
print(efficientnet)

Loaded pretrained weights for efficientnet-b4
네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 18
EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_pa

# target model의 출력 크기를 변경하여 줍니다.
efficientnet._fc = torch.nn.Linear(in_features=1792, out_features=CLASS_NUM, bias=True)

# 새롭게 넣은 네트워크 가중치를 xavier uniform으로 초기화
torch.nn.init.xavier_uniform_(efficientnet._fc.weight)
stdv = 1.0/np.sqrt(1792)
efficientnet._fc.bias.data.uniform_(-stdv, stdv)

print('네트워크 출력 채널 개수 (예측 class type 개수)', efficientnet._fc.weight.shape[0])

In [7]:
for param in efficientnet.parameters():
    param.requires_grad = False

In [8]:
for param in efficientnet._fc.parameters():
    param.requires_grad = True

### requires_grad = False 적용 됐는지 확인

In [9]:
list(efficientnet._conv_stem.parameters())[0]

Parameter containing:
tensor([[[[ 2.2086e-02, -1.4601e-02, -1.4642e-01],
          [-3.9927e-02,  2.7087e-01, -6.9427e-02],
          [-1.4240e-01, -5.8645e-03, -6.1465e-02]],

         [[-3.6127e-02,  3.9114e-02, -7.1202e-02],
          [ 1.8207e-02,  1.8403e-01, -3.8430e-02],
          [-1.9055e-02, -5.2229e-02, -1.4411e-03]],

         [[ 5.1300e-01,  5.9736e-01,  1.3116e-01],
          [ 5.7881e-01,  8.2564e-01,  2.3474e-01],
          [ 1.2164e-01,  2.0959e-01,  1.6570e-02]]],


        [[[-3.5441e-01, -5.6113e-01, -2.4834e-01],
          [-5.9000e-01, -1.0227e+00, -4.4231e-01],
          [-1.8646e-01, -4.0086e-01, -1.2527e-01]],

         [[-7.1290e-02, -1.0281e-01,  2.5308e-02],
          [-7.0111e-02, -1.8227e-01, -8.1482e-02],
          [-2.7824e-02, -4.0853e-02,  1.1460e-02]],

         [[ 4.1660e-01,  5.8368e-01,  2.2571e-01],
          [ 6.6643e-01,  1.0674e+00,  4.8188e-01],
          [ 2.2297e-01,  4.2248e-01,  1.9224e-01]]],


        [[[ 2.9867e-01,  4.5799e-02, -1.0174

In [10]:
list(efficientnet._fc.parameters())[0]

Parameter containing:
tensor([[ 0.0015, -0.0154,  0.0115,  ...,  0.0098, -0.0036, -0.0204],
        [-0.0100, -0.0016,  0.0183,  ..., -0.0178,  0.0023,  0.0157],
        [ 0.0015, -0.0109, -0.0231,  ...,  0.0141, -0.0203, -0.0191],
        ...,
        [ 0.0180,  0.0223,  0.0055,  ...,  0.0025, -0.0132, -0.0059],
        [-0.0183,  0.0061,  0.0066,  ..., -0.0102,  0.0129,  0.0103],
        [ 0.0147, -0.0026, -0.0081,  ..., -0.0190, -0.0158,  0.0021]],
       requires_grad=True)

## 2. Train Dataset 정의

In [11]:
class MyDataset(Dataset):
    def __init__(self, df, transform):
        self.img_paths = df['image_path']
        self.transform = transform
        self.y = df['target']

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.img_paths.iloc[idx])
        label = self.y.iloc[idx]

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

In [12]:
df = pd.read_csv(os.path.join(train_dir, 'train_with_label.csv'))

In [13]:
df.head()

,mask,path,image_path,age,gender,age_band,target
0,Incorrect,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,45,female,>= 30 and < 60,10
1,Wear,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,45,female,>= 30 and < 60,4
2,Wear,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,45,female,>= 30 and < 60,4
3,Not Wear,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,45,female,>= 30 and < 60,16
4,Wear,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,45,female,>= 30 and < 60,4


In [14]:
train_df, valid_df = train_test_split(df, test_size=0.3, stratify=df['target'], shuffle=True, random_state=2021)
train_df.shape, valid_df.shape

((13230, 7), (5670, 7))

In [15]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.CenterCrop(CROP_IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.56778676, 0.52758572, 0.50513912], 
                         [0.23344479, 0.24593298, 0.2498421])
])

In [16]:
train_dataset = MyDataset(train_df, transform)
valid_dataset = MyDataset(valid_df, transform)
len(train_dataset), len(valid_dataset)

(13230, 5670)

## 3. Train DataLoader 정의

In [17]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=4)

## 4. Criterion & Optimizer 정의

In [18]:
efficientnet = efficientnet.to(device)

# weight = torch.tensor([1., 1., 4] * 6).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet.parameters(), lr=LEARNING_RATE)

## 5. Train

In [19]:
%%time

val_acc_history = []
val_loss_history = []

best_model_wts = deepcopy(efficientnet.state_dict())
best_acc = 0.0

for epoch in range(1, EPOCHS + 1):
    print('Epoch {}/{}'.format(epoch, EPOCHS ))
    print('-' * 20)
    
    # Train phase
    efficientnet.train()
    
    train_running_loss = 0
    train_running_corrects = 0
    for X_batch, y_batch in tqdm(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.LongTensor)
        
        optimizer.zero_grad()
        
        y_pred = efficientnet.forward(X_batch)
        _, preds = torch.max(y_pred, 1)
        loss = criterion(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        train_running_loss += loss.item()
        train_running_corrects += torch.sum(preds == y_batch.data)

    train_epoch_loss = train_running_loss / len(train_loader)
    train_epoch_acc = train_running_corrects / len(train_loader)
    
    print(f'Train Loss : {train_epoch_loss:.4f}, Accuracy : {train_epoch_acc:.4f}')
    
    # Validation pahse
    efficientnet.eval()
    
    valid_running_loss = 0
    valid_running_corrects = 0
    with torch.no_grad():
        for X_batch, y_batch in tqdm(valid_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.LongTensor)
            
            y_pred = efficientnet.forward(X_batch)
            _, preds = torch.max(y_pred, 1)
            loss = criterion(y_pred, y_batch)
            
            valid_running_loss += loss.item()
            valid_running_corrects += torch.sum(preds == y_batch.data)
            
    valid_epoch_loss = valid_running_loss / len(valid_loader)
    valid_epoch_acc = valid_running_corrects / len(valid_loader)
    
    print(f'Validation Loss : {valid_epoch_loss:.4f}, Accuracy : {valid_epoch_acc:.4f}')   
    
    if valid_epoch_acc > best_acc:
        best_acc = valid_epoch_acc
        best_model_wts = deepcopy(efficientnet.state_dict())
    
    val_acc_history.append(valid_epoch_acc)
    val_loss_history.append(valid_epoch_loss)

print('Best Validation Accuracy: {:.4f}'.format(best_acc))
print('학습 종료!')

Epoch 1/20
--------------------



Train Loss : 1.6741, Accuracy : 73.9327



Validation Loss : 1.6057, Accuracy : 73.6000
Epoch 2/20
--------------------



Train Loss : 1.0205, Accuracy : 91.9904



Validation Loss : 0.8892, Accuracy : 95.5111
Epoch 3/20
--------------------


...


Validation Loss : 0.5534, Accuracy : 102.6222
Epoch 19/20
--------------------



Train Loss : 0.5348, Accuracy : 104.1538



Validation Loss : 0.5525, Accuracy : 103.0000
Epoch 20/20
--------------------



Train Loss : 0.5372, Accuracy : 103.8942



Validation Loss : 0.5500, Accuracy : 103.1111
Best Validation Accuracy: 103.1111
학습 종료!
CPU times: user 6min 30s, sys: 4min 43s, total: 11min 13s
Wall time: 12min 43s


## 6. Inference

In [20]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 6.1 Test Dataset 정의

In [21]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.56778676, 0.52758572, 0.50513912], 
                         [0.23344479, 0.24593298, 0.2498421])
])

In [22]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [26]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
# model = efficientnet
efficientnet.load_state_dict(best_model_wts)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_{}.csv'.format(
    datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'))), index=False)
print('test inference is done!')


test inference is done!
